# Synodal Bible & Mother

## Summary

In [ ]:
# Code summary of below

In [ ]:
!python --version

## Import

### Libraries

In [ ]:
from backend import *
import multiprocessing as mp
from multiprocessing import Pool, Process
num_cores = int(mp.cpu_count()-2)
print("Number of cores : ", mp.cpu_count())

In [ ]:
%matplotlib inline

### Definitions

In [ ]:
texts = '../texts/fiction/utf8/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', \
             'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

In [ ]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')
bibleTokenDf = pd.read_pickle('./proc/BibleTokenDf.pkl')

In [ ]:
# add stopword boolean, True if all attribute columns are null
motherTokenDf['stopword'] = ~motherTokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (motherTokenDf.shape[0]+1)), name='id')
motherTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

bibleTokenDf['stopword'] = ~bibleTokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (bibleTokenDf.shape[0]+1)), name='id')
bibleTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

## Bible and Mother Token Dataframes

In [ ]:
bibleTokenDf

In [ ]:
motherTokenDf#.set_index(['p_id','token_id'])

In [ ]:
motherTokenDf.lemma.loc[motherTokenDf.stopword == False]

In [ ]:
lemmaDf = bibleTokenDf[['lemma']].loc[bibleTokenDf.stopword == False].join(motherTokenDf[['lemma']].loc[motherTokenDf.stopword == False], on='id', lsuffix='_bible', rsuffix='_mother')
lemmaDf

## Shared Lemmas

In [ ]:
bible_lemmas = lemmaDf.lemma_bible.unique()
mother_lemmas = lemmaDf.lemma_mother.unique()

In [ ]:
common_lemmas = []
common_lemmas_count = 0
for lemma in mother_lemmas:
    #print(lemma)
    if lemma in bible_lemmas:
        common_lemmas.append(lemma)
        common_lemmas_count += 1
        
print(f"Shared unique words between Mother and the Bible: "+str(common_lemmas_count)+", that's ~"+str(round(((common_lemmas_count/len(lemmaDf.lemma_mother.unique()))*100),2))+"%")

In [ ]:
bibleTokenWindow = np.array([])
bibleTokenWindow = np.array([np.array(bibleTokenDf.lemma[i:i+7]) for i in range(len(bibleTokenDf.lemma)-6)])
bibleTokenWindow

In [ ]:
motherTokenWindow = np.array([])
motherTokenWindow = np.array([np.array(motherTokenDf.lemma[i:i+7]) for i in range(len(motherTokenDf.lemma)-6)])
motherTokenWindow

In [ ]:
def token_window(lemma_array, window_size):
    token_window_array = np.array([])
    token_window_array = np.array([np.array(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def get_sim_count(child_window_array, parent_window_array, order_matters=False):
    work_sim_count = 0
    sim_count_tuples = []
    for child_window in child_window_array:
        #print(f"child window: {child_window}")
        child_window_sim_total_count = 0
        for parent_window in parent_window_array:
            #print(f"parent window: {parent_window}")
            child_window_sim_sub_count = 0
            for i in range(len(child_window)):
                if order_matters:
                    if child_window[i] == parent_window[i]:
                        child_window_sim_total_count=+1
                        work_sim_count=+1
                else:
                    if child_window[i] in parent_window:
                        #print(f"{child_window[i]} is in parent!")
                        child_window_sim_total_count=+1
                        work_sim_count=+1
                    #else:
                        #print(f"{child_window[i]} is not in parent!")
        #print(f"{child_window}: {child_window_sim_total_count}")
        sim_count_tuples.append((child_window, child_window_sim_total_count))
    return sim_count_tuples

def dna_test_windows(child_lemmas, parent_lemmas, window_size=10):
    sim_counter = 0
    sim_list = np.array([])
    parent_window_array = token_window(parent_lemmas, window_size)
    child_window_array = token_window(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    return sim_list

In [ ]:
def ListSplit(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

In [ ]:
(dna_test_windows, [(x, bibleTokenDf.lemma) for x in ListSplit(motherTokenDf[:1000].lemma, num_cores, False)])

In [ ]:
%%time
if __name__ == "__main__":
    pool = Pool(processes=num_cores)
    workerAssignments = list(ListSplit(motherTokenWindow, num_cores, False))
    print(f"Splitting into {len(workerAssignments)} groups to make token table")
    token_start_time = time.time()
    pool.starmap(dna_test_windows, [(x, bibleTokenDf.lemma) for x in ListSplit(motherTokenDf[:1000].lemma, num_cores, False)])
    token_end_time = time.time()
    print(f"Finished in {round(token_end_time-token_start_time, 3)} seconds.")

In [ ]:
%%time
if __name__ == "__main__":
    p = Process(target=dna_test_windows, args=(motherTokenDf[:5000].lemma.loc[motherTokenDf.stopword == False], bibleTokenDf[:5000].lemma.loc[bibleTokenDf.stopword == False], 10,))
    p.start()
    p.join()

In [ ]:
%%time
sim_tuples = dna_test_windows(child_lemmas=motherTokenDf[:5000].lemma.loc[motherTokenDf.stopword == False], parent_lemmas=bibleTokenDf[:5000].lemma.loc[bibleTokenDf.stopword == False], window_size=10)

In [ ]:
pd.DataFrame(sim_tuples, columns=['window_vocab', 'sim_count']).sort_values(by='sim_count', axis=0, ascending=False)

In [ ]:
import pickle
with open('./proc/sim_tuples.pkl', 'wb') as file:
    pickle.dump(sim_tuples, file)

In [ ]:
pd.DataFrame(sim_tuples, columns=['window_vocab', 'sim_count']).locsort_values(by='sim_count', axis=0, ascending=False)

How do Gor'kii's *Mother* and the Synodal Bible overlap? 
windows of 3-5 words with 1-2+ of overlap